In [7]:
import sys
import os
sys.path.append(os.path.abspath("../src"))

In [8]:
"""
Validation loss: 2.8789, trained on ~59.7B FineWeb-Edu tokens.
This is a base model and can frequently hallucinate.
"""

import json

from time import time

import torch
import torch.nn.functional as F
import tiktoken
from torch import nn

from safetensors import safe_open

from model_550m import FlashSTU, FlashSTUConfig, get_spectral_filters


CHECKPOINT_PATH = "../models/model_step-114000.safetensors"
CONFIG_PATH = "../models/config_2-7b.json"

In [14]:
def apply_compile(model: nn.Module) -> None:
    """
    Apply torch.compile to each layer. This makes compilation efficient
    due to repeated structure. Alternatively, one can just compile the whole model.
    """
    print(f"Compiling each {model.__class__.__name__} layer with torch.compile...")
    start = time.perf_counter()
    for idx, layer in model.layers.named_children():
        compiled_layer = torch.compile(layer, mode="max-autotune", fullgraph=True)
        model.layers.register_module(idx, compiled_layer)
    end = time.perf_counter()
    print(f"Finished compiling each {model.__class__.__name__} layer in {end - start:.4f} seconds.")


def load_stu_model(config_path: str, checkpoint_path: str, device: torch.device):
    with open(config_path, "r") as f:
        config_data = json.load(f)

    dim = config_data["dim"]
    num_heads = config_data["num_heads"]
    num_layers = config_data["num_layers"]
    num_eigh = config_data["num_eigh"]
    seq_len = config_data["seq_len"]
    use_hankel_L = config_data["use_hankel_L"]
    window_size = config_data["window_size"]
    vocab_size = config_data["vocab_size"]
    mlp_scale = config_data["mlp_scale"]
    bias = config_data["bias"]
    dropout = config_data["dropout"]
    softcap = config_data["softcap"]
    theta = config_data["theta"]
    torch_compile = config_data["torch_compile"]
    torch_dtype = getattr(torch, config_data["torch_dtype"])

    model_config = FlashSTUConfig(
        dim=dim,
        num_heads=num_heads,
        num_layers=num_layers,
        seq_len=seq_len,
        window_size=window_size,
        vocab_size=vocab_size,
        mlp_scale=mlp_scale,
        bias=bias,
        dropout=dropout,
        softcap=softcap,
        theta=theta,
        torch_dtype=torch_dtype,
    )

    spectral_filters = get_spectral_filters(seq_len, num_eigh, use_hankel_L, device, torch_dtype)
    model = FlashSTU(model_config, spectral_filters)
    model = model.to(device=device, dtype=torch_dtype)

    print(f"Loading checkpoint from {checkpoint_path}...")
    state_dict = {}
    start_time = time()

    if checkpoint_path.endswith(".safetensors"):
        with safe_open(checkpoint_path, framework="pt", device=device.type) as f:
            for k in f.keys():
                state_dict[k] = f.get_tensor(k)
    elif checkpoint_path.endswith(".pt"):
        state_dict = torch.load(checkpoint_path, map_location="cpu")
    else:
        raise ValueError(f"Unsupported checkpoint format: {checkpoint_path}")
    print(f"Checkpoint loaded in {time() - start_time:.2f} seconds.")

    model.load_state_dict(state_dict, strict=True)
    print("Model weights loaded successfully!")

    if torch_compile:
        model = apply_compile(model)

    model.eval()

    return model, config_data


In [15]:
def generate_text(
    model,
    tokenizer,
    prompt,
    num_return_sequences=1,
    max_length=512,
    device="cuda",
    temperature=1.0,
    top_k=50,
):
    """
    Generate text from the given prompt using top-k sampling.

    Args:
        model: The FlashSTU model instance.
        tokenizer: The tokenizer used for encoding/decoding.
        prompt (str): Input prompt text.
        num_return_sequences (int): How many sequences to return.
        max_length (int): Maximum length of generated tokens.
        device: torch device.
        temperature (float): Sampling temperature. Higher = more random.
        top_k (int): Top-K sampling parameter.

    Returns:
        list[str]: A list of generated text sequences.
    """
    model.eval()

    # Encode prompt tokens.
    tokens = torch.tensor(
        [tokenizer.encode(prompt, allowed_special={"<|endoftext|>"})],
        device=device,
    )
    tokens = tokens.repeat(num_return_sequences, 1)

    sample_rng = torch.Generator(device=device)
    sample_rng.manual_seed(1746)

    eos_token_id = tokenizer.encode(
        "<|endoftext|>", allowed_special={"<|endoftext|>"}
    )[0]

    with torch.no_grad():
        for _ in range(max_length - tokens.size(1)):
            with torch.amp.autocast(device_type="cuda", dtype=torch.bfloat16):
                # Fwd pass. Inspect logits here.
                logits = model(tokens)     # shape: [batch, seq, vocab]
                logits = logits[:, -1, :]  # last token logits

                # Apply temperature scaling.
                if temperature > 0:
                    logits = logits / temperature

            # Compute probabilities.
            probs = F.softmax(logits, dim=-1)

            # Top-K sampling.
            top_k_probs, top_k_indices = torch.topk(probs, top_k, dim=-1)
            ix = torch.multinomial(top_k_probs, 1, generator=sample_rng)
            next_token = torch.gather(top_k_indices, -1, ix)

            # Append next token.
            tokens = torch.cat((tokens, next_token), dim=1)

            # Stop if EOS token is generated.
            if (next_token == eos_token_id).any():
                break

    # Decode all sequences.
    generated_sequences = []
    for i in range(num_return_sequences):
        decoded = tokenizer.decode(tokens[i].tolist())
        generated_sequences.append(decoded)

    return generated_sequences

In [16]:
from tiktoken.load import load_tiktoken_bpe
bpe_path = "../models/o200k_base.tiktoken"
bpe_dict = load_tiktoken_bpe(bpe_path)

tokenizer = tiktoken.Encoding(
    name="o200k_base",  # Name of the encoding
    pat_str=r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+""",
    mergeable_ranks=bpe_dict,
    special_tokens={
        "<|endoftext|>": 199999,  # Custom special token example (modify as needed)
        "<|endofprompt|>": 200018,
    }
)


In [18]:
device = torch.device("cuda")

# Load model and config.
model, config_data = load_stu_model(CONFIG_PATH, CHECKPOINT_PATH, device)
# tokenizer = tiktoken.get_encoding("o200k_base") #need to cache this for offline



AttributeError: 'STU' object has no attribute 'M_filters'

In [ ]:
# Collect prompt(s) from user.
prompts = []
while True:
    prompt = input("Enter a prompt (press enter with no text to finish): ")
    if not prompt:
        break
    prompts.append(prompt.strip())

    if len(prompts) == 0:
        print("No prompts provided. Exiting.")
        break
    

# -------------------------------------------------------------------
# BASE SETTINGS:
BASE_TEMPERATURE = 0.7  # Increase for more randomness.
BASE_TOP_K = 50         # Limit sampling to the top k tokens.
MAX_LENGTH = 512        # Maximum number of tokens to generate.
# -------------------------------------------------------------------

total_tokens = 0
start_time = time()

for i, prompt in enumerate(prompts, 1):
    print(f"Generating text for prompt {i}: {prompt}")
    generated_texts = generate_text(
        model,
        tokenizer,
        prompt,
        num_return_sequences=1,
        max_length=MAX_LENGTH,
        device=device,
        temperature=BASE_TEMPERATURE,
        top_k=BASE_TOP_K,
    )
    for gen_text in generated_texts:
        print(f"\nPrompt: {prompt}")
        print(f"Generated Text: {gen_text}\n")
        total_tokens += len(
            tokenizer.encode(gen_text, allowed_special={"<|endoftext|>"})
        )
end_time = time()
tokens_per_second = total_tokens / (end_time - start_time)
print(f"Tokens per second: {tokens_per_second:.2f}")



In [14]:
import copy
for i in range(0, 12, 2):
    stu_layer = copy.deepcopy(model.layers[i].stu)
    # stu_layer.stu_filters = stu_layer.stu_filters.to(torch.float)
    # stu_layer.M_inputs = stu_layer.M_inputs.to(torch.float)
    # torch.save(stu_layer.state_dict(), f"../stu_layers/stu_layer_{i}_60Btokens_param.pt")
    torch.save(stu_layer, f"../stu_layers/stu_layer_{i}_550m_param_full.pt")

In [19]:
from torch.amp import autocast
from torch.nn import CrossEntropyLoss
import tqdm 

from dataloader import DataLoader

val_loader = DataLoader(
    bsz=1,
    seq_len=512, 
    dataset='../fineweb-edu', 
    split="val", 
    main_process=True,
)

def evaluate(model, val_steps = 5):
    loss_fn = CrossEntropyLoss()
    val_loss = 0.0
    torch_dtype = getattr(torch, 'bfloat16')
    # val_steps = 5 # Abitrarily set to reduce long evaluations, >20 typically used
    model.eval()
    val_loader.reset()
    with torch.no_grad():
        for i, batch in tqdm.tqdm(zip(range(val_steps), val_loader, strict=False)):
            inputs, targets = batch
            inputs, targets = inputs.to(device), targets.to(device)
            if torch_dtype != torch.float32:
                with autocast(device_type=device.type, dtype=torch_dtype, cache_enabled=True):
                    preds = model(inputs)
            else:
                preds = model(inputs)

            loss = loss_fn(preds.flatten(0, 1), targets.flatten(0, 1))
            loss = loss / val_steps
            val_loss += loss.detach().float()
    return(val_loss)

2025-02-09 03:40:50,004 - INFO - Found 1 shards for split val


In [24]:
evaluate(model, val_steps = 50)

50it [00:05,  9.68it/s]


tensor(2.9617, device='cuda:0')

In [42]:
from lds import LDS

lds_layers = []
for layer in [0,2,4,6,8,10]:
    lds = LDS(10000, 896, 896, 5).to(device)
    state_dict = torch.load(f"lds_layer_{layer}_10k_5.pth")
    lds.load_state_dict(state_dict)
    lds_layers.append(lds)

/tmp/ipykernel_684723/4049127919.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f"lds_layer_{layer}_10k_5.pth")


In [49]:
for i in range(1):
    model.layers[i].stu = lds_layers[i]

In [50]:
evaluate(model, val_steps = 50)

#layer 1 is a success

50it [00:07,  6.75it/s]


tensor(2.9656, device='cuda:0')

In [52]:
model

FlashSTU(
  (tok_emb): Embedding(200064, 896)
  (dropout): Dropout(p=0.1, inplace=False)
  (layers): ModuleList(
    (0): STULayer(
      (stu_norm): RMSNorm((896,), eps=None, elementwise_affine=True)
      (stu): LDS()
      (mlp_norm): RMSNorm((896,), eps=None, elementwise_affine=True)
      (mlp): MLP(
        (gate_proj): Linear(in_features=896, out_features=10752, bias=False)
        (up_proj): Linear(in_features=896, out_features=10752, bias=False)
        (down_proj): Linear(in_features=10752, out_features=896, bias=False)
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (1): AttentionLayer(
      (attn_norm): RMSNorm((896,), eps=None, elementwise_affine=True)
      (attn): Attention(
        (c_attn): Linear(in_features=896, out_features=2688, bias=False)
        (c_proj): Linear(in_features=896, out_features=896, bias=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (mlp_norm): RMSNorm((896,), eps=None, elementwise_affine=True)
    